We will be building on top of this notebook (TODO: update link) and we will learn that under the same hyperparameter configurations [EfficientNet](https://arxiv.org/pdf/1905.11946) (NST) and its [Noisy Student](https://arxiv.org/abs/1911.04252) variant perform much better than our good old ResNet50. 

## Initial setup and imports

In [ ]:
from tf.keras.applications.efficientnet import decode_predictions
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

We first download a sample image which we will perturb in order to fool our pre-trained image classifier. We also download a dictionary that contains the mapping between the Imagenet dataset class labels and their ids (the ResNet50 model is pre-trained on ImageNet).

In [ ]:
!wget https://adversarial-ml-tutorial.org/introduction/pig.jpg
!wget -q https://raw.githubusercontent.com/USCDataScience/dl4j-kerasimport-examples/master/dl4j-import-example/data/imagenet_class_index.json

## A few early constants

In [ ]:
EPS = 2./255
IMAGENET_LABELS = "imagenet_class_index.json"

with open(IMAGENET_LABELS) as f:
    IMAGENET_CLASSES = {int(i):x[1] for i,x in json.load(f).items()}

## Utility functions

Next we define a couple of utility functions that we will be using throughout the course of this notebook.

In [ ]:
# Function to load up an image from a local filepath and
# display it
def show_image(image_path):
    image_pixels = plt.imread(image_path)
    plt.imshow(image_pixels)
    plt.show()
    return image_pixels

# Function to preprocess an image for performing inference
def preprocess_image(image_pixels):
    image_pixels = cv2.resize(image_pixels, (224, 224))
    image_pixels = np.expand_dims(image_pixels, axis=0)

    return image_pixels

# Clipping utility to project delta back to our favorable pixel
# range ([0 ,1])
def clip_eps(delta_tensor):
    return tf.clip_by_value(delta_tensor, 
                            clip_value_min=-EPS, 
                            clip_value_max=EPS)

# Parse the label
def get_label(preds):
    print(IMAGENET_CLASSES[preds.argmax()])

## Downloading the noisy student training weights and preparing them

In [ ]:
!wget -q https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/noisystudent/noisy_student_efficientnet-b0.tar.gz
!tar -xf noisy_student_efficientnet-b0.tar.gz

!wget -q https://raw.githubusercontent.com/yixingfu/tensorflow/updateweights/tensorflow/python/keras/applications/efficientnet_weight_update_util.py
!python efficientnet_weight_update_util.py --model b0 --ckpt \
       noisy_student_efficientnet-b0/model.ckpt --o efficientnetb0.h5

## Loading our first image and pre-trained models

In [ ]:
sample_image = show_image("pig.jpg")
enb0 = tf.keras.applications.EfficientNetB0(weights="imagenet")
enb0_nst = tf.keras.applications.EfficientNetB0(weights="efficientnetb0.h5")

## Running inference with the pre-trained models

In [ ]:
# Run inference EfficientNetB0 (ImageNet)
preds = enb0.predict(preprocessed_image)
print("Predicted:"", decode_predictions(preds, top=3)[0])
print("Predicted:", decode_predictions(preds, top=3)[0])
print("Class idx:", preds.argmax())

In [ ]:
# Run inference EfficientNetB0 (NST)
preds = enb0_nst.predict(preprocessed_image)
print("Predicted:"", decode_predictions(preds, top=3)[0])
print("Predicted:", decode_predictions(preds, top=3)[0])
print("Class idx:", preds.argmax())

## Utility functions for adversarial attacks

In this attack we will use PGD to simply increase the loss for the given class such that the visual semantics of our input image does not get hampered.

In [ ]:
def generate_adversaries(image_tensor, delta, model, true_class_index):
    # Loss and optimizer
    scc_loss = tf.keras.losses.SparseCategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

    for t in range(50):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            inp = image_tensor + delta
            predictions = model(inp, training=False)
            loss = - scc_loss(
                    tf.convert_to_tensor([true_class_index]),
                    predictions
                )
            if t % 5 == 0:
                print(t, loss.numpy())
            
        # Get the gradients
        gradients = tape.gradient(loss, delta)
        
        # Update the weights
        optimizer.apply_gradients([(gradients, delta)])

        # Project the delta back (l-infinite norm)
        delta.assign_add(clip_eps(delta))

    return delta

def perturb_image(image_path, true_class_index, model):
    # Load and preprocess image but a but without any `preprocess_input`
    sample_image = show_image(image_path)
    preprocessed_image = preprocess_image(sample_image)

    # Generate predictions before any adversaries
    unsafe_preds = model.predict(preprocessed_image)
    print("Prediction before adv.:", decode_predictions(unsafe_preds, top=3)[0])

    # Initialize the perturbation quantity
    image_tensor = tf.constant(preprocessed_image, dtype=tf.float32)
    delta = tf.Variable(tf.zeros_like(image_tensor), trainable=True)

    # Get the learned delta
    delta_tensor = generate_adversaries(image_tensor, delta, model, true_class_index)

    # See if the image changes
    plt.imshow((image_tensor + delta_tensor).numpy().squeeze()/255)
    plt.show()

    # Generate prediction
    perturbed_image = image_tensor + delta_tensor
    preds = model.predict(perturbed_image)
    print("Prediction after adv.:", decode_predictions(preds, top=3)[0])

## Launching a simple attack!

In [ ]:
perturb_image("pig.jpg", 341, enb0)

In [ ]:
perturb_image("pig.jpg", 341, enb0_nst)